# Web-scraping: сбор данных из баз данных и интернет-источников

*Алла Тамбовцева*


## Практикум 5. Управление браузером с Selenium: продолжение

Импортируем всё необходимое – это все модули, библиотеки и функции, которые мы успели изучить на данный момент:

In [1]:
import re
import requests
import pandas as pd

from selenium import webdriver as wd
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

Вспомним, на чем мы остановились в прошлый раз. Запустим код, который открывает окно браузера через Selenium, переходит на главную страницу книжного магазина «Библио-Глобус», вводит в строку поиска запрос *python* и сохраняет карточки с товарами в виде объектов BeautifulSoup:

In [2]:
# чтобы все можно было запускать сразу (не в отдельных ячейках)
# и не беспокоиться о том, что страница не прогрузится,
# добавляем время ожидания через implicitly_wait()

br = wd.Chrome()
br.maximize_window()

br.get("https://www.biblio-globus.ru/")
br.implicitly_wait(5)

search = br.find_element(By.ID, "SearchBooks")
search.send_keys("python")
br.implicitly_wait(2)

search.send_keys(Keys.ENTER)
br.implicitly_wait(2)

options = br.find_element(By.TAG_NAME, "select")
options.send_keys(" Сначала дешевле ")
tick = br.find_element(By.CLASS_NAME, "custom-control-label")
tick.click()
br.implicitly_wait(2)

html = br.page_source
soup = BeautifulSoup(html)
cards = soup.find_all("div", class_ = "card")

### Задача 0

Для бодрого начала сделайте скриншот той страницы, которая сейчас открыта в окне браузера, запущенном через Selenium, используя 
метод `.save_screenshot()`.

In [3]:
# сохраняем в файл test.png
br.save_screenshot("test.png")

True

### Задача 1

Напишите функцию `get_cards()`, которая принимает на вход строку с кодом HTML, соответствующим странице с результатами запроса (пока это первая страница результатов), и возвращает список объектов BeautifulSoup с карточками товаров (как `cards` выше). 

In [4]:
type(cards[0]) # изучаем тип данных

bs4.element.Tag

In [5]:
def get_cards(html):
    soup = BeautifulSoup(html)
    cards = soup.find_all("div", class_ = "card")
    return cards

### Задача 2

Найдите фрагмент (объект типа `WebElement` в Selenium) с кнопкой, которая позволяет переходить на *следующую* страницу поиска, и сохраните его в переменную `next_page`. Найдите фрагмент с кнопкой , которая позволяет переходить на *последнюю* страницу поиска, и сохраните его в переменную `last_page`.

In [6]:
# поиск по частичному совпадению текста ссылки (» или »»)

next_page = br.find_element(By.PARTIAL_LINK_TEXT, "»")
last_page = br.find_element(By.PARTIAL_LINK_TEXT, "»»")

In [7]:
# полезные примеры – как понять, что внутри объекта WebElement
# innerHTML – какой код HTML внутри фрагмента last_page

last_page.get_attribute("innerHTML")

'<span aria-hidden="true">»»</span><span class="sr-only">Next</span>'

In [8]:
# outerHTML – какой код HTML снаружи фрагмента last_page
# в какие тэги он вложен

last_page.get_attribute("outerHTML")

'<a aria-label="Next" class="page-link" href="search?query=python&amp;page=8&amp;sort=5&amp;instock=on&amp;cat=0&amp;isdiscount="><span aria-hidden="true">»»</span><span class="sr-only">Next</span></a>'

### Задача 3

Используя метод `.get_attribute()`, извлеките из `last_page` ссылку на последнюю страницу поиска, сохраните ее в переменную `last_href`.

In [9]:
last_href = last_page.get_attribute("href")

In [10]:
print(last_href)

https://www.biblio-globus.ru/catalog/search?query=python&page=8&sort=5&instock=on&cat=0&isdiscount=


### Задача 4

Используя регулярные выражения, извлеките из `last_href` номер последней страницы с результатами поиска и превратите результат в целое число (тип `integer`).

In [11]:
last_n = int(re.search("page=(\d+)", last_href).group(1))
print(last_n)

8


### Задача 5

Допишите код ниже таким образом, чтобы все страницы с результатами поиска прокликались в открытом окне браузера, а в `full_res` оказались все карточки с товарами.

In [12]:
full_res = cards.copy()

for i in range(0, last_n - 1):
    
    next_page = br.find_element(By.PARTIAL_LINK_TEXT, "»")
    next_page.click()
    
    br.implicitly_wait(2)
    
    html = br.page_source
    cards_to_add = get_cards(html)
    full_res.extend(cards_to_add)

In [13]:
print(len(full_res)) # примерное число результатов

94


### Задача 6

Изучите элементы списка `full_res` и придумайте способ, который позволит универсальным образом различать карточки с товарами и «пустые» карточки с фрагментами меню. 

Подсказка: сравните атрибуты у первого и второго элемента в `full_res`.

In [ ]:
# print(full_res[0])
# print(full_res[1])

In [14]:
item0 = full_res[0]
item1 = full_res[1]

print(item0.get("class")) # 4 элемента = 4 слова внутри атрибута class
print(item1.get("class")) # 1 элемент = 1 слова внутри атрибута class

# итог: чтобы различать такие объекты, можем поставить условие на длину списка внутри class

['card', 'sidebar-menu', 'mb-4']
['card']


### Задача 7

Опираясь на решение предыдущей задачи, сохраните в список `full_clean` только те элементы, которые содержат информацию о товарах.

In [15]:
full_clean = []

for item in full_res:
    if len(item.get("class")) == 1:
        full_clean.append(item)
        
print(len(full_clean)) # теперь лишнее убрано, элементов меньше

78


### Задача 8

Изучите один элемент списка `full_clean` и напишите функцию `get_info()`, которая принимает на вход карточку товара и возвращает список из следующих элементов:

* название товара (самое полное);
* ссылка на файл с изображением товара;
* цена товара (в текстовом формате с указанием валюты);
* полная ссылка на страницу с полным описанием товара.

In [16]:
# самое полное название – в альтернативном тексте к изображению (атрибут alt в тэге <img>)
# ссылка на изображение – в источнике изображения (атрибут src в тэге <img>)
# далее ищем подходящие разделы и ссылки, как обычно

def get_info(item):
    
    img = item.find("img")
    name = img.get("alt")
    image_link = img.get("src")
    price = item.find("div", class_ = "price_item_block").text
    link = item.find("a", class_ = "img-wrapper-index").get("href")
        
    return name, price, link, image_link

### Задача 9

Примените функцию `get_info()` ко всем элементам из `full_clean`, соберите информацию о всех товарах и сохраните ее в датафрейм. Скорректируйте данные или типы столбцов в датафрейме при необходимости. Экспортируйте данные в файл Excel.

In [17]:
L = [get_info(item) for item in full_clean]
df = pd.DataFrame(L)
df.head()

,0,1,2,3
0,"Быстрый доступ. Python: советы, функции, подск...",399 ₽,/product/11015030,https://static1.bgshop.ru/imagehandler.ashx?fi...
1,Тетрадь общая с пластиковой обложкой на спирал...,419 ₽,/product/10657321,https://static1.bgshop.ru/imagehandler.ashx?fi...
2,Python для непрограммистов. Самоучитель в прим...,479 ₽,/product/10992579,https://static1.bgshop.ru/imagehandler.ashx?fi...
3,Python глазами хакера,509 ₽,/product/10823441,https://static1.bgshop.ru/imagehandler.ashx?fi...
4,Практикум по анализу данных на языках Python и R,509 ₽,/product/10882364,https://static1.bgshop.ru/imagehandler.ashx?fi...


In [18]:
# добавляем названия столбцов
df.columns = ["title", "price", "link", "image_link"]

In [19]:
# преобразуем текст с ценой и делаем столбец числовым
df["price"] = df["price"].str.strip(" ₽").astype(int)

In [20]:
df.dtypes # все ок

title         object
price          int64
link          object
image_link    object
dtype: object

In [21]:
df.head()

,title,price,link,image_link
0,"Быстрый доступ. Python: советы, функции, подск...",399,/product/11015030,https://static1.bgshop.ru/imagehandler.ashx?fi...
1,Тетрадь общая с пластиковой обложкой на спирал...,419,/product/10657321,https://static1.bgshop.ru/imagehandler.ashx?fi...
2,Python для непрограммистов. Самоучитель в прим...,479,/product/10992579,https://static1.bgshop.ru/imagehandler.ashx?fi...
3,Python глазами хакера,509,/product/10823441,https://static1.bgshop.ru/imagehandler.ashx?fi...
4,Практикум по анализу данных на языках Python и R,509,/product/10882364,https://static1.bgshop.ru/imagehandler.ashx?fi...


### Задача 10

Используя полученный в предыдущей задаче датафрейм, создайте список пар *номер строки в датафрейме*-*ссылка на изображения товаров*. Используя функцию ниже, сохраните в папку `imgs` файлы с изображениями товаров. Названия файлов должны содержать номер строки в 
датафрейме, например, `1.jpg`.

In [23]:
def download_img(img_link, file_name):
    """
    img_link: link to the image (str)
    file_name: name of file to save the image (str)
    """
    img_data = requests.get(img_link).content
    with open(file_name, 'wb') as file:
        file.write(img_data)

In [22]:
pairs = list(zip(list(df.index), list(df["image_link"].values)))

# пример пары
print(pairs[0])

(0, 'https://static1.bgshop.ru/imagehandler.ashx?fileName=11015030.jpg&width=200')


Импортируем модуль `os` для работы с системой (*operating system*):

In [24]:
import os

In [26]:
# создаем папку imgs внутри текущей рабочей папки

os.makedirs("imgs") 

In [27]:
# перед индексом строки index доклеиваем imgs/, чтобы сохранение происходило в эту папку
# после него доклеиваем расширение файлов .jpg
# после запуска кода в папке imgs должны появиться файлы с картинками (фото товаров)

for index, link in pairs:
    name = "imgs/" + str(index) + ".jpg"
    download_img(link, name)